### Django - продолжение
### Создание моделей

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 

#### Источник: https://developer.mozilla.org/ru/docs/Learn/Server-side/Django/Models

#### Пусть стоит задача хранить информацию о книгах - локальная библиотека 

In [2]:
Image(url='UML.png')

### 0..* 1..* - означает, что, например, какому-то жанру могут соответствовать 0 и более книг. А книге соответствует 1 и более жанров

#### Пример модели:

In [6]:
from django.db import models

class MyModelName(models.Model):
    """
    A typical class defining a model, derived from the Model class.
    """

    # Fields
    my_field_name = models.CharField(max_length=20, help_text="Enter field documentation")
    ...

    # Metadata
    class Meta:
        ordering = ["-my_field_name"]

    # Methods
    def get_absolute_url(self):
         """
         Returns the url to access a particular instance of MyModelName.
         """
        return reverse('model-detail-view', args=[str(self.id)])

    def __str__(self):
        """
        String for representing the MyModelName object (in Admin site etc.)
        """
        return self.field_name

#### поля = столбцы данных, которые будут в нашей БД (my_field_name..)

In [7]:
Image(url='help.png')

### Порядок, в котором объявляются поля, будет влиять на их порядок по умолчанию, если модель отображается в форме (например, на сайте администратора), хотя это может быть переопределено.

In [8]:
Image(url='args_field.png')

In [9]:
Image(url='types_of_field.png')

### Полный список полей: https://docs.djangoproject.com/en/1.10/ref/models/fields/#field-types

### Метаданные

In [2]:
class Meta:
    ordering = ["-my_field_name"]

In [3]:
ordering = ["title", "-pubdate"] 
verbose_name = "BetterName"

### Книги будут отсортированы по названию от A-Z и по дате публикации от новых к старым

### Полный список атрибутов метаданных: https://docs.djangoproject.com/en/2.2/ref/models/options/#model-meta-options

### -------------------------------------
### Методы

In [5]:
def get_absolute_url(self):
     """
     Returns the url to access a particular instance of MyModelName.
     """
    return reverse('model-detail-view', args=[str(self.id)])

def __str__(self):
    """
    String for representing the MyModelName object (in Admin site etc.)
    """
    return self.field_name

### --------------------------------------
### Теперь модель для нашей библиотеки: нужно запихнуть ее в models.py

In [ ]:
from django.db import models

### сначала зададим модель для жанра
class Genre(models.Model):
    ## жанр 
    name = models.CharField(max_length=200, help_text="Enter a book genre (e.g. Science Fiction, French Poetry etc.)")
    def __str__(self):
        return self.name
### Языковая модель 
class Language(models.Model):
    lang = models.CharField(max_length=100)
    def __str__(self):
        return self.lang
    
    
    
from django.urls import reverse #Used to generate URLs by reversing the URL patterns
### модель книги
class Book(models.Model):
    """
    Model representing a book (but not a specific copy of a book).
    """
    title = models.CharField(max_length=200)
    author = models.ForeignKey('Author', on_delete=models.SET_NULL, null=True)
    # Foreign Key used because book can only have one author, but authors can have multiple books
    # Author as a string rather than object because it hasn't been declared yet in the file.
    summary = models.TextField(max_length=1000, help_text="Enter a brief description of the book")
    isbn = models.CharField('ISBN',max_length=13, help_text='13 Character <a href="https://www.isbn-international.org/content/what-isbn">ISBN number</a>')
    genre = models.ManyToManyField(Genre, help_text="Select a genre for this book")
    # ManyToManyField used because genre can contain many books. Books can cover many genres.
    # Genre class has already been defined so we can specify the object above.
    language = models.ManyToManyField(Language, help_text='Введите язык')
    def __str__(self):
        """
        String for representing the Model object.
        """
        return self.title


    def get_absolute_url(self):
        """
        Returns the url to access a particular book instance.
        """
        return reverse('book-detail', args=[str(self.id)])
###
###
###
### Класс экземпляра книги:
import uuid # Required for unique book instances

class BookInstance(models.Model):
    """
    Model representing a specific copy of a book (i.e. that can be borrowed from the library).
    """
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, help_text="Unique ID for this particular book across whole library")
    book = models.ForeignKey('Book', on_delete=models.SET_NULL, null=True)
    imprint = models.CharField(max_length=200) 
    due_back = models.DateField(null=True, blank=True) ### поле для времени, через которое книга снова появится для взятия

    LOAN_STATUS = (
        ('m', 'Maintenance'),
        ('o', 'On loan'),
        ('a', 'Available'),
        ('r', 'Reserved'),
    )

    status = models.CharField(max_length=1, choices=LOAN_STATUS, blank=True, default='m', help_text='Book availability')

    class Meta:
        ordering = ["due_back"]


    def __str__(self):
        """
        String for representing the Model object
        """
        return '%s (%s)' % (self.id,self.book.title)
    
###
###
###
### Класс автора ( модель)
class Author(models.Model):
    """
    Model representing an author.
    """
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    date_of_birth = models.DateField(null=True, blank=True)
    date_of_death = models.DateField('Died', null=True, blank=True)

    def get_absolute_url(self):
        """
        Returns the url to access a particular author instance.
        """
        return reverse('author-detail', args=[str(self.id)])


    def __str__(self):
        """
        String for representing the Model object.
        """
        return '%s, %s' % (self.last_name, self.first_name)
###
###


In [17]:
def func():
    return '(%s (%s))' % ('Первое слово','Второе слово')
func()

'(Первое слово (Второе слово))'

### ----------------------------------------
### Панель администратора

### В файл admin.py залить:

In [ ]:
from .models import Author, Genre, Book, BookInstance, Language

admin.site.register(Book)
admin.site.register(Author)
admin.site.register(Genre)
admin.site.register(BookInstance)
admin.site.register(Language)

### Создание суперпользователя 

In [ ]:
python3 manage.py createsuperuser

### Изменение отображения в админ-панели

In [ ]:
### комментируем строчку в admin.py
# admin.site.register(Author)
### создаем класс 
class AuthorAdmin(admin.ModelAdmin):
    pass

### комментируем admin.site.register(Book) и admin.site.register(BookInstance)

# Register the admin class with the associated model
admin.site.register(Author, AuthorAdmin)
@admin.register(Book) ### то же самое, что и admin.site.register...
class BookAdmin(admin.ModelAdmin):
    pass

# Register the Admin classes for BookInstance using the decorator

@admin.register(BookInstance)
class BookInstanceAdmin(admin.ModelAdmin):
    pass

In [ ]:
### изменяем
class AuthorAdmin(admin.ModelAdmin):
    list_display = ('last_name', 'first_name', 'date_of_birth', 'date_of_death')

### Текущий вид admin.py:

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import Author, Genre, Book, BookInstance, Language

# admin.site.register(Book)
# admin.site.register(Author)
admin.site.register(Genre)
# admin.site.register(BookInstance)
admin.site.register(Language)

class AuthorAdmin(admin.ModelAdmin):
    list_display = ('last_name', 'first_name', 'date_of_birth', 'date_of_death')

@admin.register(Book) ### то же самое, что и admin.site.register...
class BookAdmin(admin.ModelAdmin):
    pass

@admin.register(BookInstance)
class BookInstanceAdmin(admin.ModelAdmin):
    pass
    
admin.site.register(Author, AuthorAdmin)
admin.site.register(Book, BookAdmin)
admin.site.register(BookInstance, BookInstanceAdmin)

### добавим в models.py к модели Book: 

In [ ]:

def display_genre(self):
        """
        Creates a string for the Genre. This is required to display genre in Admin.
        """
        return ', '.join([ genre.name for genre in self.genre.all()[:3] ])
    display_genre.short_description = 'Genre'

### .....
### Попробовать самому: 
### Для представления списка BookInstance , добавьте код для отображения книги, статуса, даты возврата, и id (вместо значения по умолчанию возвращаемого __str__() ).
### Добавьте встроенный список перечня Book в представление списка Author , используя тот же самый подход, который мы применили для Book/BookInstance.

### -------------------------
### Теперь создание домашенй страницы

In [8]:
Image(url= "HTTP request.png")

### Будем выстраивать переходы от urls.py к views.py и templates

In [9]:
Image(url= "urlsToDo.png")

### Вставить это в catalog/urls.py вместо пустого urlpatterns

In [11]:
urlpatterns = [
    path('', views.index, name='index'),
]

### Вставить это в views.py в catalog'е

In [ ]:
from .models import Book, Author, BookInstance, Genre

def index(request):
    """
    Функция отображения для домашней страницы сайта.
    """
    # Генерация "количеств" некоторых главных объектов
    num_books=Book.objects.all().count()
    num_instances=BookInstance.objects.all().count()
    # Доступные книги (статус = 'a')
    num_instances_available=BookInstance.objects.filter(status__exact='a').count()
    num_authors=Author.objects.count()  # Метод 'all()' применён по умолчанию.

    # Отрисовка HTML-шаблона index.html с данными внутри
    # переменной контекста context
    return render(
        request,
        'index.html',
        context={'num_books':num_books,'num_instances':num_instances,'num_instances_available':num_instances_available,'num_authors':num_authors},
    )

### создадим новый файл base_generil.html в catalog/templates/

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>

  {% block title %}<title>Local Library</title>{% endblock %}
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>

  <!-- Добавление дополнительного статического CSS файла -->
  {% load static %}
  <link rel="stylesheet" href="{% static 'css/styles.css' %}">
</head>

<body>

  <div class="container-fluid">

    <div class="row">
      <div class="col-sm-2">
      {% block sidebar %}
      <ul class="sidebar-nav">
          <li><a href="{% url 'index' %}">Home</a></li>
          <li><a href="">All books</a></li>
          <li><a href="">All authors</a></li>
      </ul>
     {% endblock %}
      </div>
      <div class="col-sm-10 ">
      {% block content %}{% endblock %}
      </div>
    </div>

  </div>
</body>
</html>

### создадим файл по пути catalog/static/css/styles.css с содержимым:


In [ ]:
.sidebar-nav {
    margin-top: 20px;
    padding: 0;
    list-style: none;
}

### создадим исходную index.html страницу в пути catalog/templates/index.html с содержимым:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
<h1>Local Library Home</h1>

  <p>Welcome to <em>LocalLibrary</em>, a very basic Django website developed as a tutorial example on the Mozilla Developer Network.</p>

<h2>Dynamic content</h2>

  <p>The library has the following record counts:</p>
  <ul>
    <li><strong>Books:</strong> {{ num_books }}</li>
    <li><strong>Copies:</strong> {{ num_instances }}</li>
    <li><strong>Copies available:</strong> {{ num_instances_available }}</li>
    <li><strong>Authors:</strong> {{ num_authors }}</li>
  </ul>

{% endblock %}

### -----------------------------------
### Страница со списком книг

### путь до страницы будет catalog/books 

### дополняем urls.py: добавляем строчку, связанную с запросом books и import url

In [ ]:
from django.urls import path, re_path, include
from . import views

urlpatterns = [
    re_path(r'^$', views.index, name='index'),
    re_path(r'^books/$', views.BookListView.as_view(), name='books'),
]

### копируем это в views.py в catalog'е

In [ ]:
from django.views import generic

class BookListView(generic.ListView):
    model = Book

### ВАЖНО! Теперь, нужно создать шаблон по пути catalog/templates/catalog/book_list.html . Тут 2 раза catalog фигурирует

### можем переопределить метод, взятый из стандратной библиотеки

In [ ]:
class BookListView(generic.ListView):
    model = Book

    def get_queryset(self):
        return Book.objects.filter(title__icontains='war')[:5] # Получить 5 книг, содержащих 'war' в заголовке

### создаем шаблон catalog/templates/catalog/book_list.html с содержимым:


In [ ]:
{% extends "base_generic.html" %}

{% block content %}
    <h1>Book List</h1>

    {% if book_list %}
    <ul>

      {% for book in book_list %}
      <li>
        <a href="{{ book.get_absolute_url }}">{{ book.title }}</a> ({{book.author}})
      </li>
      {% endfor %}

    </ul>
    {% else %}
      <p>There are no books in the library.</p>
    {% endif %}
{% endblock %}

### идем в base_generic.html и там делаем ссылку на нашу страницу book_list таким образом: меняем строчку с All books на это (наделяем ссылкой):

In [ ]:
<li><a href="{% url 'books' %}">All books</a></li>

### теперь сделаем для каждой книги свою страничку, так чтобы url адрес был: catalog/book/<id> 

### В catalog urls.py дополняем строчку с book_detail в urlpatterns

In [ ]:

urlpatterns = [
    re_path(r'^$', views.index, name='index'),
    re_path(r'^books/$', views.BookListView.as_view(), name='books'),
    re_path(r'^book/(?P<pk>\d+)$', views.BookDetailView.as_view(), name='book-detail'),
]

### В отличие от предыдущих преобразований, в данном случае мы применяем наше регулярное выражение (РВ) для сопоставления "настоящего паттерна", а не просто строки. Данное РВ сопоставляет любой URL-адрес, который начинается с book/, за которым до конца строки (до маркера конца строки - знак доллара) следуют одна, или более цифр. В процессе выполнения данного преобразования, оно "захватывает" цифры и передаёт их в функцию отображения как параметр с именем pk.


### регулярное выражение: r'<ваше регулярное выражение>'

In [8]:
Image(url='regular.png')

In [7]:
Image(url='patterns2.png')

### Вставляем в catalog/urls.py эту часть кода 

In [ ]:
class BookDetailView(generic.DetailView):
    model = Book

### создаем шаблон /locallibrary/catalog/templates/catalog/book_detail.htm детальной информации

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Title: {{ book.title }}</h1>

  <p><strong>Author:</strong> <a href="">{{ book.author }}</a></p> <!-- author detail link not yet defined -->
  <p><strong>Summary:</strong> {{ book.summary }}</p>
  <p><strong>ISBN:</strong> {{ book.isbn }}</p>
  <p><strong>Language:</strong> {{ book.language }}</p>
  <p><strong>Genre:</strong> {% for genre in book.genre.all %} {{ genre }}{% if not forloop.last %}, {% endif %}{% endfor %}</p>

  <div style="margin-left:20px;margin-top:20px">
    <h4>Copies</h4>

    {% for copy in book.bookinstance_set.all %}
    <hr>
    <p class="{% if copy.status == 'a' %}text-success{% elif copy.status == 'd' %}text-danger{% else %}text-warning{% endif %}">{{ copy.get_status_display }}</p>
    {% if copy.status != 'a' %}<p><strong>Due to be returned:</strong> {{copy.due_back}}</p>{% endif %}
    <p><strong>Imprint:</strong> {{copy.imprint}}</p>
    <p class="text-muted"><strong>Id:</strong> {{copy.id}}</p>
    {% endfor %}
  </div>
{% endblock %}

In [ ]:
### примечательно: 
{% for copy in book.bookinstance_set.all %}
<!-- итерации по каждой копии/экземпляру книги -->
{% endfor %}

### добавим в модель Author в catalog/models.py: 


In [ ]:
class Meta:
    ordering = ['last_name']

### Вуаля!

### добавим это внизу base_generic.html сразу после block content endblock


In [ ]:
{% block content %}{% endblock %}

{% block pagination %}
  {% if is_paginated %}
      <div class="pagination">
          <span class="page-links">
              {% if page_obj.has_previous %}
                  <a href="{{ request.path }}?page={{ page_obj.previous_page_number }}">previous</a>
              {% endif %}
              <span class="page-current">
                  Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}.
              </span>
              {% if page_obj.has_next %}
                  <a href="{{ request.path }}?page={{ page_obj.next_page_number }}">next</a>
              {% endif %}
          </span>
      </div>
  {% endif %}
{% endblock %}

### по аналогии можно добавить author list и страничку об авторе